In [212]:
import pandas as pd
import json
from pathlib import Path
from datetime import datetime
import os

In [213]:
data_dir = os.getcwd()
data_dir

'c:\\Users\\nirma\\Projects\\Github\\Spotify-Historical-PowerBI\\Data'

In [214]:
# Get all JSON files in the Data directory
import glob
json_files = glob.glob(os.path.join(data_dir, "*.json"))
print(f"Found {len(json_files)} JSON files in {data_dir}")

all_records = []

for file_path in json_files:
    try:
        print(f"Processing: {os.path.basename(file_path)}")
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            all_records.extend(data)
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON format in {file_path}")
    except Exception as e:
        print(f"Error processing {file_path}: {str(e)}")

print(f"Total records loaded: {len(all_records)}")

Found 8 JSON files in c:\Users\nirma\Projects\Github\Spotify-Historical-PowerBI\Data
Processing: Streaming_History_Audio_2014-2017_0.json
Processing: Streaming_History_Audio_2017-2018_1.json
Processing: Streaming_History_Audio_2018-2019_2.json
Processing: Streaming_History_Audio_2019-2021_3.json
Processing: Streaming_History_Audio_2021-2023_5.json
Processing: Streaming_History_Audio_2021_4.json
Processing: Streaming_History_Audio_2023-2025_6.json
Processing: Streaming_History_Audio_Test.json
Total records loaded: 104397


In [215]:
spotify_df = pd.DataFrame(all_records)
spotify_df.drop(columns=['episode_name', 'episode_show_name', 'ip_addr', 'incognito_mode', 'offline_timestamp', 'offline', 'audiobook_title', 'audiobook_uri','audiobook_chapter_uri','audiobook_chapter_title', 'spotify_episode_uri'], inplace=True, errors='ignore')
#spotify_df.info()
spotify_df['ts'] = pd.to_datetime(spotify_df['ts'])

In [216]:
spotify_df[['platform', 'ts']].drop_duplicates().sort_values("ts", ascending=False).to_csv('platform.csv', index=False)
#spotify_df['platform'] = spotify_dfvalue_counts().plot(kind='bar', title='Platform Distribution', figsize=(10, 5))

In [217]:
spotify_df.columns

Index(['ts', 'platform', 'ms_played', 'conn_country',
       'master_metadata_track_name', 'master_metadata_album_artist_name',
       'master_metadata_album_album_name', 'spotify_track_uri', 'reason_start',
       'reason_end', 'shuffle', 'skipped'],
      dtype='object')

In [218]:
def standardize_platforms(df, platform_col='platform', new_col='platform_standardized', inplace=True, print_stats=True):
    """
    Standardize platform names in the given DataFrame.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame containing platform information
    platform_col : str, default 'platform'
        Name of the column containing platform information
    new_col : str, default 'platform_standardized'
        Name of the new column to create with standardized platforms
    inplace : bool, default False
        Whether to modify the DataFrame in-place or return a copy
    print_stats : bool, default True
        Whether to print statistics about the standardized platforms
    
    Returns:
    --------
    pandas.DataFrame
        DataFrame with added column containing standardized platform names
    pandas.Series
        Platform distribution counts
    """
    # Create a copy if not modifying in-place
    if not inplace:
        df = df.copy()
    
    # Initialize the new column with original platform values
    df[new_col] = df[platform_col]
    
    # Standardize platforms using regex pattern matching
    platform_mapping = {
        # Pattern: standardized name
        '^iOS|^ios$': 'iOS',
        'Windows|windows|web_player|desktop|Linux': 'Windows',
        '(Google, Pixel XL)': 'Google Pixel',
        'Echo|not_applicable': 'Amazon Echo',
        'Tesla': 'Tesla',
        'cast|tizen|ps4|SCEI': 'Android TV'
    }
    
    # Apply each mapping pattern
    for pattern, platform_name in platform_mapping.items():
        df.loc[df[platform_col].str.contains(pattern, case=False, na=False), new_col] = platform_name
    
    # Calculate platform distribution
    platform_counts = df[new_col].value_counts()
    
    # Print statistics if requested
    if print_stats:
        print("Unique standardized platforms:")
        print(df[new_col].drop_duplicates().sort_values().values)
        
        print("\nPlatform distribution:")
        print(platform_counts)
    
    return df, platform_counts

# Apply the standardization function to the DataFrame
spotify_df, platform_counts = standardize_platforms(spotify_df)

C:\Users\nirma\AppData\Local\Temp\ipykernel_96620\3747657065.py:45: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df.loc[df[platform_col].str.contains(pattern, case=False, na=False), new_col] = platform_name


Unique standardized platforms:
['Amazon Echo' 'Android TV' 'Google Pixel'
 'Partner android_sdk GOOGLE;pixel_xl;f87c03896d274ecf9d80f86e942202e1;9_arm64-v8a'
 'Tesla' 'Windows' 'android' 'iOS']

Platform distribution:
platform_standardized
iOS                                                                                 36504
Amazon Echo                                                                         31615
Windows                                                                             23375
Google Pixel                                                                        11172
Android TV                                                                            790
android                                                                               726
Tesla                                                                                 214
Partner android_sdk GOOGLE;pixel_xl;f87c03896d274ecf9d80f86e942202e1;9_arm64-v8a        1
Name: count, dtype: int64


In [219]:
spotify_df.drop(columns=['platform'], inplace=True, errors='ignore')
spotify_df.rename(columns={
    'platform_standardized': 'platform',
    'conn_country': 'country',
    'master_metadata_track_name': 'track_name',
    'master_metadata_album_artist_name': 'artist_name',
    'master_metadata_album_album_name': 'album_name',
    },inplace=True)

In [220]:
def format_text_columns(df, columns, inplace=False):
    """
    Perform basic text formatting on specified columns in a DataFrame.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame containing columns to format
    columns : list of str
        Column names to apply text formatting
    inplace : bool, default False
        Whether to modify the DataFrame in-place or return a copy
    
    Returns:
    --------
    pandas.DataFrame
        DataFrame with formatted text columns
    """
    # Create a copy if not modifying in-place
    if not inplace:
        df = df.copy()
    
    for column in columns:
        if column in df.columns:
            # Only process non-null values
            mask = df[column].notna()
            
            # Step 1: Remove leading/trailing whitespace
            df.loc[mask, column] = df.loc[mask, column].str.strip()
            
            # Step 2: Fix multiple spaces
            df.loc[mask, column] = df.loc[mask, column].str.replace(r'\s+', ' ', regex=True)
            
            # Step 3: Apply title case (capitalize first letter of each word)
            df.loc[mask, column] = df.loc[mask, column].str.title()
    
    return df

# Apply text formatting to track, artist, and album name columns
columns_to_format = ['track_name', 'artist_name', 'album_name']
spotify_df = format_text_columns(spotify_df, columns_to_format, inplace=True)

# Show a sample of the formatted data
print("Sample of formatted text columns:")
spotify_df[columns_to_format].sample(5)

Sample of formatted text columns:


,track_name,artist_name,album_name
34165,"Tera Hone Laga Hoon (From ""Ajab Prem Ki Ghazab...",Atif Aslam,Ajab Prem Ki Ghazab Kahani (Original Motion Pi...
101027,Ferxxoko,Joyce Santana,Ferxxoko
81365,Lovely (With Khalid),Billie Eilish,Lovely (With Khalid)
2732,Animals,Maroon 5,V
28642,Xo,Keywest,Cover Sessions - The Ladies Vol.1


In [221]:
# Verify text formatting and check for any issues
print(f"Number of unique tracks: {spotify_df['track_name'].nunique()}")
print(f"Number of unique artists: {spotify_df['artist_name'].nunique()}")
print(f"Number of unique albums: {spotify_df['album_name'].nunique()}")

# Check for any unusually long titles
long_titles = spotify_df[spotify_df['track_name'].str.len() > 50].sort_values(by='track_name')
if len(long_titles) > 0:
    print(f"\nFound {len(long_titles)} tracks with names longer than 50 characters:")
    print(long_titles[['track_name']].head())

Number of unique tracks: 23604
Number of unique artists: 9751
Number of unique albums: 17864

Found 2722 tracks with names longer than 50 characters:
                                              track_name
22486  "We Circle Through The Night, Consumed By Fire...
22484  (What'S So Funny Bout) Peace, Love And Underst...
28129  (What'S So Funny Bout) Peace, Love And Underst...
23511  (What'S So Funny Bout) Peace, Love And Underst...
23306  (You Make Me Feel Like) A Natural Woman - Reco...


In [222]:
spotify_df.to_csv(os.path.join(data_dir, 'spotify_data_cleaned.csv'), index=False)
# Save the cleaned DataFrame to a CSV file
print(f"Cleaned data saved to {os.path.join(data_dir, 'spotify_data_cleaned.csv')}")

Cleaned data saved to c:\Users\nirma\Projects\Github\Spotify-Historical-PowerBI\Data\spotify_data_cleaned.csv
